In this notebook, we train the Chinese language text-only classification models for Humanitarian Classification Task

We install libraries needed for the task.

In [1]:
!pip install tweet-preprocessor
!pip install transformers
!pip install ftfy

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 46.9 MB/s 
     |████████████████████████████████| 6.5 MB 13.9 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 53 kB 1.5 MB/s 


We use the [CrisisMMD](https://arxiv.org/abs/1805.00713) dataset comprising of 7216 Twitter posts (images + text) that are categorized into 5 humanitarian categories. We store the dataset (eg filename for DEV set : `task_humanitarian_text_img_dev.tsv`) on a Google Drive at the (relative) path `/content/drive/My Drive/crisis_bert/data/` and mount the drive at this path. We are assuming that this notebook is stored at the path /content/drive/My Drive/crisis_bert/code

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/crisis_bert/code')
!pwd

Mounted at /content/drive
/content/drive/My Drive/crisis_bert/code


For each of the train, test and dev files, we preprocess the data and and pickle and store the preprocessed files at `/content/drive/My Drive/crisis_bert/data`

In [3]:
import numpy as np
import pandas as pd
import os
import sys
import pickle as pkl
import preprocessor as tp
import ftfy

def FormatText(filename):
    data_list = []
    data = pd.read_csv(filename, sep = '\t')

    # Clean the "tweet_text" column
    tp.set_options(tp.OPT.URL, tp.OPT.EMOJI, tp.OPT.SMILEY, tp.OPT.RESERVED)
    data["tweet_text"] = data["tweet_text"].apply(lambda x: tp.clean(x))
    data["tweet_text"] = data["tweet_text"].apply(lambda x : ftfy.fix_text(x))
    data["tweet_text"] = data["tweet_text"].str.replace(r'\\n',' ', regex=True) 
    data["tweet_text"] = data["tweet_text"].str.replace(r"\'t", " not")
    data["tweet_text"] = data["tweet_text"].str.strip()
    data["tweet_text"] = data["tweet_text"].str.replace("#","")
    data["tweet_text"] = data["tweet_text"].str.replace("@","")
    tweet_id = data['tweet_id'].to_list()
    image_id = data['image_id'].to_list()
    tweet_text = data['tweet_text'].to_list()
    tweet_text = [str(x) for x in tweet_text]

    label = data['label'].to_list()
    alignment = data['label_text_image'].to_list()
    for a_var in range(len(tweet_id)):
        data_point = {}
        if alignment[a_var] == 'Positive':
            data_point['tweet_id'] = tweet_id[a_var]
            data_point['image_id'] = image_id[a_var]
            data_point['tweet_text'] = tweet_text[a_var]
            data_point['label'] = label[a_var] #these labels are not final, need to be updated in a downstream script
            data_list.append(data_point)
    return data_list

folderpath = '../data/'
filenames = ['task_humanitarian_text_img_dev.tsv', 'task_humanitarian_text_img_test.tsv', 'task_humanitarian_text_img_train.tsv']

for a_file in filenames:
    data = FormatText(folderpath + a_file)
    source_text = [x['tweet_text'] for x in data]
    for a_var in range(len(data)):
        data[a_var]['tweet_text'] = [source_text[a_var]]
    with open(folderpath + a_file.split('.')[0] + '.pkl', 'wb') as f:
        pkl.dump(data, f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


We use these pickled files to convert the preprocessed data to a suitable input for building and training the model.

In [4]:
import os, sys, re
import torch
import pickle as pkl
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertForMaskedLM, BertTokenizer

NUM_LABELS = 5

class DatasetFormatting(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def AlignFormatLabels(list, labels):
    aligned_list = []
    for item in list:
        if item == 'vehicle_damage':
            aligned_list.append('infrastructure_and_utility_damage')
        elif item == 'missing_or_found_people' or item == 'injured_or_dead_people':
            aligned_list.append('affected_individuals')
        else:
            aligned_list.append(item)
    final_labels = []
    for item in aligned_list:
        final_labels.append(labels.index(item))
    return final_labels

def GetData(folder):
    filenames_prefix = 'task_humanitarian_text_img_'
    with open(folder + filenames_prefix + 'train.pkl', 'rb') as f:
        train_data = pkl.load(f)
    with open(folder + filenames_prefix + 'dev.pkl', 'rb') as f:
        val_data = pkl.load(f)
    with open(folder + filenames_prefix + 'test.pkl', 'rb') as f:
        test_data = pkl.load(f)
    train_texts = []
    train_labels = []
    train_ids = []
    for a_point in train_data:
        # print(a_point)
        train_ids.append(a_point['tweet_id'])
        train_texts.append(a_point['tweet_text'])
        train_labels.append(a_point['label'])
    val_texts = []
    val_labels = []
    val_ids = []
    for a_point in val_data:
        val_ids.append(a_point['tweet_id'])
        val_texts.append(a_point['tweet_text'])
        val_labels.append(a_point['label'])
    test_texts = []
    test_labels = []
    test_ids = []
    for a_point in test_data:
        test_ids.append(a_point['tweet_id'])
        test_texts.append(a_point['tweet_text'])
        test_labels.append(a_point['label'])
    my_labels = ['other_relevant_information', 'affected_individuals', 'rescue_volunteering_or_donation_effort', 'infrastructure_and_utility_damage', 'not_humanitarian']
    
    train_texts = [x[0] for x in train_texts]
    val_texts = [x[0] for x in val_texts]
    test_texts = [x[0] for x in test_texts]
    train_labels = AlignFormatLabels(train_labels, my_labels)
    val_labels = AlignFormatLabels(val_labels, my_labels)
    test_labels = AlignFormatLabels(test_labels, my_labels)
    return train_texts, train_labels, train_ids, val_texts, val_labels, val_ids, test_texts, test_labels, test_ids

pickle_folder_path = '../data/'
train_texts, train_labels, train_ids, val_texts, val_labels, val_ids, test_texts, test_labels, test_ids = GetData(pickle_folder_path)

In [5]:
import pandas as pd 
train_df = pd.DataFrame({'text': train_texts,'label': train_labels})
eval_df = pd.DataFrame({'text': val_texts,'label': val_labels})
test_df = pd.DataFrame({'text': test_texts,'label': test_labels})
train_df.head(5)

,text,label
0,KAKEnews: California wildfires destroy more th...,3
1,KAKEnews: California wildfires destroy more th...,3
2,KAKEnews: California wildfires destroy more th...,3
3,TheAtlantic: Photos of California's destructiv...,3
4,Why California's wildfires are worse in the fall.,0


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import sklearn

tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = DatasetFormatting(train_encodings, train_labels)
val_dataset = DatasetFormatting(val_encodings, val_labels)
test_dataset = DatasetFormatting(test_encodings, test_labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    print(pred.predictions)
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

For demo purposes, we train only for 2 epochs.

In [11]:
#comment this line to enable logging of weights and biases 
#(you might need a wandb account and an API key for this!)
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='./results_',          # output directory
    num_train_epochs=2 ,             # total number of training epochs
    per_device_train_batch_size=8,  # batchx size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # learning_rate = 5e-5
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    save_steps = 500,
    evaluation_strategy='epoch'
)

model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm', num_labels=NUM_LABELS)

model.config.output_hidden_states = True

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    tokenizer = tokenizer,
    # compute_metrics=compute_metrics,
    eval_dataset=val_dataset             # evaluation dataset
)

# Training
trainer.train()
# trainer.train()
trainer.save_model('trained_models')


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly i

Epoch,Training Loss,Validation Loss
1,0.900500,0.808998
2,0.560200,0.641484


Saving model checkpoint to ./results_/checkpoint-500
Configuration saved in ./results_/checkpoint-500/config.json
Model weights saved in ./results_/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results_/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results_/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 998
  Batch size = 16
Saving model checkpoint to ./results_/checkpoint-1000
Configuration saved in ./results_/checkpoint-1000/config.json
Model weights saved in ./results_/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results_/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results_/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results_/checkpoint-1500
Configuration saved in ./results_/checkpoint-1500/config.json
Model weights saved in ./results_/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results_/checkpoint-1500/token

In [12]:
trainer.save_model('trained_models')
torch.save(model.state_dict(), 'trained_models_new/trained_model_chinese.pt')

Saving model checkpoint to trained_models
Configuration saved in trained_models/config.json
Model weights saved in trained_models/pytorch_model.bin
tokenizer config file saved in trained_models/tokenizer_config.json
Special tokens file saved in trained_models/special_tokens_map.json


In [14]:
probs, _, metrics = trainer.predict(test_dataset)
pred_labels = probs[0].argmax(-1)

***** Running Prediction *****
  Num examples = 955
  Batch size = 16


In [15]:
from sklearn.metrics import classification_report
target_names=['other_relevant_information', 'affected_individuals', 'rescue_volunteering_or_donation_effort', 'infrastructure_and_utility_damage', 'not_humanitarian']
print(classification_report(test_labels, pred_labels ,target_names=target_names))

                                        precision    recall  f1-score   support

            other_relevant_information       0.77      0.73      0.75       235
                  affected_individuals       0.00      0.00      0.00         9
rescue_volunteering_or_donation_effort       0.69      0.74      0.71       126
     infrastructure_and_utility_damage       0.68      0.64      0.66        81
                      not_humanitarian       0.82      0.85      0.83       504

                              accuracy                           0.78       955
                             macro avg       0.59      0.59      0.59       955
                          weighted avg       0.77      0.78      0.77       955



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
